In [6]:
class Parent:
    def __init__(self, value):
        self.value = value
        print(f"Parent initialized with value {self.value}")

class Child(Parent):
    def __init__(self, value, extra):
        super().__init__(value)  # 调用父类的 __init__ 方法
        self.extra = extra
        print(f"Child initialized with value {self.value} and extra {self.extra}")


In [7]:
Child.value = 10
Child.extra = 20


c=Child(10,20)

Parent initialized with value 10
Child initialized with value 10 and extra 20


## Tensor 张量

In pytorch, the core data type is tensor, which is a multi-dimensional matrix containing elements of a single data type. The core operation associated with this data type is gradient calculation. In this notebook, we will introduce some basic concepts and code operations related to it.

在PyTorch框架中的一个核心数据类型是张量，即Tensor。伴随此数据类型的核心操作是梯度计算。下面将逐一说明与之相关一些基本概念和代码操作。

### 1. Creating Tensor 张量的创建

In [2]:
import torch
import numpy as np

t0 = torch.zeros(5, 3, 2)                          #各元素用0填充
t1 = torch.ones(5, 3, 2)                           #各元素用1填充
tr = torch.randn(5, 3, 2)                          #各元素用标准正态分布随机数填充
lis2t = torch.tensor([[1,2],[3,4],[5,6]])          # 直接由列表数据构造张量
ta = torch.arange(12.0)                            # 由浮点型序列构造张量
arr2t = torch.tensor(np.array([[1,2],[3,4]]))      # 将np数组转化成张量
mat2t = torch.tensor(np.matrix([[1,3],[3,5]]))     # 将np矩阵转换成张量
t2mat = np.matrix(mat2t)                           # 将张量转换成np矩阵
t2arr = np.array(arr2t)                            # 将张量转换成np数组


### 2. Tensor Attributes 张量的属性

In [3]:
s0 = t0.shape                                      # 张量的形状
device0 = t0.device                                # 张量的运算设备（如GPU或CPU）
print(t0.requires_grad)                            # 当前张量是否会被视为自变量来计算梯度
print(t0.is_leaf)                                  # 当前张量是否为叶子节点（即手创或由“不计算梯度”张量运算出的张量）
print(t0.grad_fn)                                  # 生成当前张量的生成函数（凡叶子节点的生成函数都为None）


False
True
None


### 3. Tensor Operations 张量的操作

In [4]:
ele = tr[0][1][1].item()                           # 取张量中的一个元素。注意：只有标量才能取其值
tta = ta.reshape(2, 2, 3)                          # 更改张量形状，但元素数要与原张量一致
arr2ft = arr2t.float()                             # 将张量中各元素类型转为浮点型，许多张量运算只适用于浮点型
y1 = arr2ft * mat2t                                # 得到两张量的对应元素乘积
y2 = arr2ft @ arr2ft                               # 对2阶张量（矩阵）做点积
y3 = torch.t(arr2t)                                # 对2阶以下张量求其转置
y4 = torch.inverse(arr2ft)                         # 对2阶张量求其逆张量（张量中元素必须是浮点型）
m2 = y2.mean()                                     # 求张量中各元素均值（也只适用于浮点型元素）


### 4. Tensor Gradient 张量的梯度

In [7]:
arr2ft.requires_grad = True                        # 将要计算梯度的张量的“计算梯度”属性置为真，默认是假
yy = arr2ft @ arr2ft                               # 做矩阵乘法
yy.requires_grad                                   # “计算梯度”属性置真后算得的张量，其该属性也自动为真
print(yy.is_leaf)                                  # 由“计算梯度”张量运算出的新张量不再是叶子节点
print(arr2ft.is_leaf)                              # 手创的“计算梯度”张量是叶子节点
print(yy.grad_fn)                                  # 非叶子张量会有生成函数
fy = yy.mean()                                     # 由张量运算出标量，因为只能针对作为标量的最终函数值计算梯度
print(arr2ft.grad)                                 # 梯度计算前叶子张量的梯度属性为空
fy.backward()                                      # 利用偏导的反向传递计算各叶子张量的梯度
print(arr2ft.grad)                                 # 每次算得的梯度会累加进叶子张量的梯度属性里


False
True
None
tensor([[-0.3750,  0.3750],
        [-0.3750,  0.3750]])


In PyTorch framework, no matter how complex the function operation is, whether it is a library function or a handwritten one, as long as a scalar function value can be obtained in the end, the gradient of this function value to each leaf tensor can be traced back. The mathematical principle behind it is the partial derivative calculation formula of multivariate functions.

在PyTorch框架中，各叶子张量无论经历怎样复杂的函数运算，这些函数也无论是库函数还是手写的，只要最终能得到一个标量函数值，都能追溯到此函数值对各叶子张量的梯度，其背后的数学原理就是多元函数的偏导计算公式。

### 5. 更新叶子张量后使其仍保持叶子属性

前面提到叶子节点必须是手创或是全由"不计算梯度"的张量运算出的张量，但在神经网络训练过程中，待训练的参数张量一方面必须要设置成是“计算梯度”的，另一方面在每次训练中要被迭代更新，而这更新过程就是由原参数张量参与计算得到新参数张量，这样一来，新参数张量必然不再是叶子节点，也就无法再进行下一次训练迭代了。为解决此问题，需要设法使更新后的叶子张量仍保持其叶子属性。此时要用到Python的上下文操作：

In [6]:
with torch.no_grad():
    arr2ft = arr2ft - arr2ft.grad                   # 在no_grad上下文中更新叶子张量
print(arr2ft.is_leaf)                               # 此时原叶子张量虽被运算更新，但仍恢复成最初叶子属性
arr2ft.requires_grad                                # 梯度计算开关也置回默认关闭状态
print(arr2ft.grad)                                  # 梯度属性也自动清空
arr2ft = arr2ft - arr2ft.grad                       # 如果直接更新叶子张量，则会将其变为生成张量，即非叶子张量
print(arr2ft.is_leaf)


True
None


TypeError: unsupported operand type(s) for -: 'Tensor' and 'NoneType'